imDB - 2k

Install Required Dependencies

In [ ]:
# Install evaluate library for load_metric function
!pip install evaluate

In [9]:
!pip install --upgrade accelerate

 First Train on Full IMDB Dataset

In [2]:
!python train_nlp.py \
    --task_name imdb \
    --model_name_or_path roberta-base \
    --output_dir ./data-model/imdb/all-data \
    --do_train \
    --train_logger \
    --num_train_epochs 3 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 512 \
    --val-index-path ./data-model/imdb/val_index.npy

^C


Step 2: Generate Importance Scores and Features

In [1]:
!python train_nlp.py \
    --task_name imdb \
    --model_name_or_path ./data-model/imdb/all-data \
    --output_dir ./data-model/imdb/all-data \
    --do_eval \
    --eval_train \
    --save_feature \
    --save_confidence \
    --save_importance_scores \
    --training_dynamics \
    --per_device_eval_batch_size 16 \
    --max_length 256

^C


06/17/2025 09:10:09 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cpu

Mixed precision type: no

Traceback (most recent call last):
  File "d:\Research\d2pruning\train_nlp.py", line 1376, in <module>
    main()
  File "d:\Research\d2pruning\train_nlp.py", line 621, in main
    raw_datasets = preprocess_for_val(args, raw_datasets, val_size=1000)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Research\d2pruning\train_nlp.py", line 187, in preprocess_for_val
    if os.path.exists(args.val_index_path):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen genericpath>", line 19, in exists
TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType


Step 3: Select 2k Examples and Train

In [ ]:
# Set parameters for 2k selection
CORESET_RATIO = 0.1  # 2.5k out of 25k examples
N_NEIGHBOR = 10
GAMMA = 0.1

!python train_nlp.py \
    --task_name imdb \
    --model_name_or_path roberta-base \
    --output_dir ./data-model/imdb/coreset-2k \
    --do_train \
    --coreset \
    --coreset-mode class \
    --budget-mode uniform \
    --sampling-mode graph \
    --data-score-path ./data-model/imdb/all-data/data-score-imdb.pickle \
    --feature-path ./data-model/imdb/all-data/train-features.npy \
    --coreset-key forgetting \
    --coreset-ratio 0.08 \
    --mis-ratio 0.4 \
    --label-balanced \
    --n-neighbor 5 \
    --num_train_epochs 3 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/imdb/val_index.npy

---

# COLA Dataset Experiment

Step 1: Train on Full COLA Dataset

In [13]:

!python train_nlp.py \
    --train_file ./data-model/cola/dataset/train \
    --validation_file ./data-model/cola/dataset/validation \
    --model_name_or_path roberta-base \
    --output_dir ./data-model/cola/all-data \
    --do_train \
    --train_logger \
    --num_train_epochs 3 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 128

Traceback (most recent call last):
  File "d:\Research\d2pruning\train_nlp.py", line 1366, in <module>
    main()
  File "d:\Research\d2pruning\train_nlp.py", line 530, in main
    args = parse_args()
           ^^^^^^^^^^^^
  File "d:\Research\d2pruning\train_nlp.py", line 518, in parse_args
    assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
AssertionError: `train_file` should be a csv or a json file.


In [ ]:
# Download and prepare COLA dataset from GLUE
import os
os.makedirs('./data-model/cola', exist_ok=True)

from datasets import load_dataset
dataset = load_dataset('glue', 'cola')
dataset.save_to_disk('./data-model/cola/dataset')
print("COLA dataset downloaded and saved successfully!")

Step 2: Generate Importance Scores and Features for COLA

In [ ]:
!python train_nlp.py \
    --train_file ./data-model/cola/dataset/train \
    --validation_file ./data-model/cola/dataset/validation \
    --model_name_or_path ./data-model/cola/all-data \
    --output_dir ./data-model/cola/all-data \
    --do_eval \
    --eval_train \
    --save_feature \
    --save_confidence \
    --save_importance_scores \
    --training_dynamics \
    --per_device_eval_batch_size 32 \
    --max_length 128

Step 3: Select Subset and Train COLA (0.5 ratio for smaller dataset)

In [ ]:
# Set parameters for COLA selection (smaller dataset, higher ratio)
CORESET_RATIO = 0.5  # 50% of COLA training data
N_NEIGHBOR = 5
GAMMA = 0.1

!python train_nlp.py \
    --task_name cola \
    --model_name_or_path roberta-base \
    --output_dir ./data-model/cola/coreset-50pct \
    --do_train \
    --coreset \
    --coreset-mode class \
    --budget-mode uniform \
    --sampling-mode graph \
    --data-score-path ./data-model/cola/all-data/data-score-cola.pickle \
    --feature-path ./data-model/cola/all-data/train-features.npy \
    --coreset-key forgetting \
    --coreset-ratio 0.5 \
    --mis-ratio 0.4 \
    --label-balanced \
    --n-neighbor 5 \
    --num_train_epochs 3 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 128

In [ ]:
# Now train on the downloaded COLA dataset
!python train_nlp.py \
    --train_file ./data-model/cola/dataset/train \
    --validation_file ./data-model/cola/dataset/validation \
    --model_name_or_path roberta-base \
    --output_dir ./data-model/cola/all-data \
    --do_train \
    --train_logger \
    --num_train_epochs 3 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 128

Check and Update .gitignore

In [2]:
# Check if .gitignore exists and contains env/
import os

gitignore_path = '.gitignore'
env_ignored = False

if os.path.exists(gitignore_path):
    with open(gitignore_path, 'r') as f:
        content = f.read()
        if 'env/' in content:
            env_ignored = True
            print("✅ .gitignore exists and env/ is already ignored")
        else:
            print("⚠️ .gitignore exists but env/ is not ignored")
else:
    print("❌ .gitignore does not exist")

if not env_ignored:
    # Create or update .gitignore
    gitignore_content = """# Python virtual environment
env/
venv/
ENV/
env.bak/
venv.bak/

# Python cache files
__pycache__/
*.py[cod]
*$py.class

# Model files and data
data-model/
*.pickle
*.pkl
*.npy
*.h5
*.pth
*.bin

# Jupyter Notebook
.ipynb_checkpoints

# IDE files
.vscode/
.idea/

# OS files
.DS_Store
Thumbs.db

# Log files
*.log
logs/
"""
    
    with open(gitignore_path, 'w') as f:
        f.write(gitignore_content)
    
    print("✅ .gitignore created/updated with env/ folder ignored")

# Double check git status
print("\n" + "="*50)
print("Current git status for env folder:")
os.system('git status env/ 2>/dev/null || echo "env/ is properly ignored or not tracked"')

✅ .gitignore exists and env/ is already ignored

Current git status for env folder:


0